In [92]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer

from google.colab import drive

In [2]:
drive.mount('/content/gdrive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [3]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database

In [4]:
!unzip '/content/gdrive/MyDrive/Kaggle/anime-recommendations-database.zip'

In [62]:
dataset_anime = pd.read_csv('/content/gdrive/MyDrive/Kaggle/anime.csv')
dataset_rating = pd.read_csv('/content/gdrive/MyDrive/Kaggle/rating.csv')

In [63]:
dataset_anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [64]:
print('Total anime dengan id yang unique: ', len(dataset_anime.anime_id.unique()))

Total anime dengan id yang unique:  12294


In [65]:
dataset_anime.isna().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [66]:
dataset_rating.isna().sum()

user_id     0
anime_id    0
rating      0
dtype: int64

In [9]:
print('Total anime yang sudah di rating: ', len(dataset_rating.anime_id.unique()))

Total anime yang sudah di rating:  11200


In [8]:
dataset_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB


In [10]:
dataset_anime.dropna(inplace=True)

In [11]:
isna_isnull_dataset_anime = pd.DataFrame({
    'isnull': dataset_anime.isnull().sum(),
    'isna':dataset_anime.isna().sum()
})
isna_isnull_dataset_anime

isnull  isna
anime_id       0     0
name           0     0
genre          0     0
type           0     0
episodes       0     0
rating         0     0
members        0     0

In [12]:
isna_isnull_dataset_rating = pd.DataFrame({
    'isnull': dataset_rating.isnull().sum(),
    'isna':dataset_rating.isna().sum()
})
isna_isnull_dataset_rating

isnull  isna
user_id        0     0
anime_id       0     0
rating         0     0

COLLABORATIVE BASE

In [13]:
dataset_anime.rename(columns={'rating':'rating_all'})

anime_id                                               name  \
0         32281                                     Kimi no Na wa.   
1          5114                   Fullmetal Alchemist: Brotherhood   
2         28977                                           Gintama°   
3          9253                                        Steins;Gate   
4          9969                                      Gintama&#039;   
...         ...                                                ...   
12289      9316       Toushindai My Lover: Minami tai Mecha-Minami   
12290      5543                                        Under World   
12291      5621                     Violence Gekiga David no Hoshi   
12292      6133  Violence Gekiga Shin David no Hoshi: Inma Dens...   
12293     26081                   Yasuji no Pornorama: Yacchimae!!   

                                                   genre   type episodes  \
0                   Drama, Romance, School, Supernatural  Movie        1   
1      Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64   
2      Action, Comedy, Historical, Parody, Samurai, S...     TV       51   
3                                       Sci-Fi, Thriller     TV       24   
4      Action, Comedy, Historical, Parody, Samurai, S...     TV       51   
...                                                  ...    ...      ...   
12289                                             Hentai    OVA        1   
12290                                             Hentai    OVA        1   
12291                                             Hentai    OVA        4   
12292                                             Hentai    OVA        1   
12293                                             Hentai  Movie        1   

       rating_all  members  
0            9.37   200630  
1            9.26   793665  
2            9.25   114262  
3            9.17   673572  
4            9.16   151266  
...           ...      ...  
12289        4.15      211  
12290        4.28      183  
12291        4.88      219  
12292        4.98      175  
12293        5.46      142  

[12017 rows x 7 columns]

In [14]:
dataset_combination = pd.merge(dataset_rating,dataset_anime[['anime_id','name']],how='left',on='anime_id')
dataset_combination = dataset_combination[dataset_combination['rating'] > 0]

In [73]:
dataset_combination = dataset_combination.drop_duplicates(['anime_id','user_id'])

- Mengubah user_id menjadi list tanpa nilai yang sama
- Melakukan encoding user_id
- Melakukan proses encoding angka ke ke user_id


In [15]:
user_ids = dataset_combination['user_id'].unique().tolist()

user_to_user_encoded = {x: i for i, x in enumerate(user_ids)}

user_encoded_to_user = {i: x for i, x in enumerate(user_ids)}

- Mengubah anime_id menjadi list tanpa nilai yang sama
- Melakukan proses encoding angka ke anime_id
- Melakukan proses encoding anime_id

In [16]:
anime_ids = dataset_combination['anime_id'].unique().tolist()

animeid_to_animeid_encoded = {x: i for i, x in enumerate(anime_ids)}

animeid_encoded_to_animeid = {i: x for i, x in enumerate(anime_ids)}

- Mendapatkan jumlah user
- Mendapatkan jumlah resto



In [17]:
total_user = len(user_to_user_encoded)
print("total user : " , total_user)

total_anime = len(animeid_to_animeid_encoded)
print("total user : " , total_anime)

total user :  69600
total user :  9927


- Mapping user_id ke dataframe user_id_encoded
- Mapping anime_id ke dataframe anime_id_encoded

In [18]:
dataset_combination['user_id_encoded'] = dataset_combination['user_id'].map(user_to_user_encoded)

dataset_combination['anime_id_encoded'] = dataset_combination['anime_id'].map(animeid_to_animeid_encoded)

- Mengubah rating menjadi nilai float

In [19]:
dataset_combination['rating'] = dataset_combination['rating'].values.astype(np.float32)

- Mengacak dataset

In [20]:
dataset_combination = dataset_combination.sample(random_state=42,n=5000)
dataset_combination

user_id  anime_id  rating                           name  \
5656992    53103      6707     7.0                Kuroshitsuji II   
3552500    32884       339     8.0        Serial Experiments Lain   
2636208    24907     23755     9.0              Nanatsu no Taizai   
5554182    52213     10156     7.0                   Sacred Seven   
95779        994      1562    10.0  Yamato Nadeshiko Shichihenge♥   
...          ...       ...     ...                            ...   
4102045    39002      2926     6.0               Myself; Yourself   
6154510    57624     10076     4.0          Kämpfer für die Liebe   
453639      4665     16001    10.0   Kokoro Connect: Michi Random   
2650701    25036      6875     9.0                       Iron Man   
1592679    15436     24703     7.0           High School DxD BorN   

         user_id_encoded  anime_id_encoded  
5656992            50245               774  
3552500            31108               523  
2636208            23560                80  
5554182            49405              2505  
95779                934               790  
...                  ...               ...  
4102045            36855               805  
6154510            54554               580  
453639              4378              1218  
2650701            23684              1603  
1592679            14582               649  

[5000 rows x 6 columns]

In [21]:
min_rating = min(dataset_combination.rating)
max_rating = max(dataset_combination.rating)

In [22]:
features = dataset_combination[['user_id_encoded','anime_id_encoded']].values
label = dataset_combination['rating'].values

x_train, x_test, y_train, y_test = train_test_split(
    features,
    label,
    test_size=0.2,
    random_state=42
)

In [23]:
scaler = MinMaxScaler()
y_train = scaler.fit_transform(y_train.reshape(-1,1))
y_test = scaler.transform(y_test.reshape(-1, 1))


In [74]:
class RecommenderNet(tf.keras.Model):

  def __init__(self, total_user, total_anime, embedding_size, **kwargs):

    super(RecommenderNet, self).__init__(**kwargs)

    self.total_user = total_user
    self.total_anime = total_anime
    self.embedding_size = embedding_size

    self.user_embedding = layers.Embedding(
        total_user,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(0.00005)
    )

    self.user_bias = layers.Embedding(total_user, 1)

    self.anime_embedding = layers.Embedding(
        total_anime,
        embedding_size,
        embeddings_initializer = 'he_normal',
        embeddings_regularizer = keras.regularizers.l2(0.005)
    )
    self.anime_bias = layers.Embedding(total_anime, 1)


    self.dropout = Dropout(0.00005)

  def call(self, inputs):

    user_vector = self.user_embedding(inputs[:,0])
    user_bias = self.user_bias(inputs[:, 0])

    anime_vector = self.anime_embedding(inputs[:, 1])
    anime_bias = self.anime_bias(inputs[:, 1])

    dot_user_anime = tf.tensordot(user_vector, anime_vector, 2)

    x = dot_user_anime + user_bias + anime_bias

    return tf.nn.sigmoid(x)

In [75]:
# inisialisasi model
model = RecommenderNet(total_user, total_anime, 50)

In [76]:
# Buat callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_root_mean_squared_error',
    mode='min',
    patience=50,
    min_delta=0.001
)


In [77]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    metrics=[
        tf.keras.metrics.RootMeanSquaredError(),
    ],
)

history = model.fit(
    x = x_train,
    y = y_train,
    epochs = 5,
    validation_data = (x_test, y_test),
    callbacks=[early_stopping],
)

Epoch 1/5
125/125 [==============================] - 9s 63ms/step - loss: 0.9603 - root_mean_squared_error: 0.3074 - val_loss: 0.8094 - val_root_mean_squared_error: 0.3099
Epoch 2/5
125/125 [==============================] - 7s 57ms/step - loss: 0.7518 - root_mean_squared_error: 0.3055 - val_loss: 0.7181 - val_root_mean_squared_error: 0.3096
Epoch 3/5
125/125 [==============================] - 7s 58ms/step - loss: 0.7031 - root_mean_squared_error: 0.3044 - val_loss: 0.6978 - val_root_mean_squared_error: 0.3092
Epoch 4/5
125/125 [==============================] - 7s 56ms/step - loss: 0.6920 - root_mean_squared_error: 0.3033 - val_loss: 0.6936 - val_root_mean_squared_error: 0.3089
Epoch 5/5
125/125 [==============================] - 7s 55ms/step - loss: 0.6887 - root_mean_squared_error: 0.3020 - val_loss: 0.6928 - val_root_mean_squared_error: 0.3085


In [78]:
user_id = dataset_rating.user_id.sample(1).iloc[0]
anime_rated_by_user = dataset_rating[dataset_rating.user_id == user_id]

anime_not_rated = dataset_anime[~dataset_anime['anime_id'].isin(anime_rated_by_user.anime_id.values)]['anime_id']
anime_not_rated = list(
    set(anime_not_rated)
    .intersection(set(animeid_to_animeid_encoded.keys()))
)

anime_not_rated = [[animeid_to_animeid_encoded.get(x)] for x in anime_not_rated]
user_encoder = user_to_user_encoded.get(user_id)
user_anime_array = np.hstack(
    ([[user_encoder]] * len(anime_not_rated), anime_not_rated)
)

In [128]:
ratings = model.predict(user_anime_array).flatten()

top_ratings_indices = ratings.argsort()[-10:][::-1]
recommended_anime_ids = [
    animeid_encoded_to_animeid.get(anime_not_rated[x][0]) for x in top_ratings_indices
]

print('Showing recommendations for users: {}'.format(user_id))
print('===' * 9)
print('Resto with high ratings from user')
print('----' * 8)

top_anime_user = (
    anime_rated_by_user.sort_values(
        by = 'rating',
        ascending=False
    )
    .head(5)
    .anime_id.values
)

anime_df_rows = dataset_anime[dataset_anime['anime_id'].isin(top_anime_user)]
for row in anime_df_rows.itertuples():
    print(row.name, ':', row.genre)

print('----' * 8)
print('Top 10 resto recommendation')
print('----' * 8)

recommended_anime = dataset_anime[dataset_anime['anime_id'].isin(recommended_anime_ids)]
for row in recommended_anime.itertuples():
    print(row.name, ':', row.genre)


194/194 [==============================] - 1s 4ms/step
Showing recommendations for users: 42635
Resto with high ratings from user
--------------------------------
Boku dake ga Inai Machi : Mystery, Psychological, Seinen, Supernatural
Tsumiki no Ie : Drama
Zankyou no Terror : Psychological, Thriller
Colorful (Movie) : Drama, Slice of Life, Supernatural
Wolf&#039;s Rain : Action, Adventure, Drama, Fantasy, Mystery, Sci-Fi
--------------------------------
Top 10 resto recommendation
--------------------------------
Love Live! The School Idol Movie : Music, School, Slice of Life
InuYasha: Kagami no Naka no Mugenjo : Adventure, Demons, Drama, Fantasy, Historical, Romance, Shounen, Supernatural
Digimon Tamers : Adventure, Comedy, Drama, Fantasy, Shounen
Lupin III: Otakara Henkyaku Daisakusen!! : Adventure, Comedy, Shounen
Toradora!: Bentou no Gokui : Comedy, Romance, School, Slice of Life
Ao no Exorcist: Ura Ex : Action, Fantasy, Shounen, Supernatural
Hayate no Gotoku! Heaven Is a Place on E

In [133]:
print(ratings.argsort()[:10])
print(ratings[:10])

[2425  835 5915 4821 1464 4200  705 3636 1390 4777]
[0.5059893  0.4968958  0.48240396 0.4909011  0.48381287 0.48594034
 0.49208197 0.49698597 0.5069916  0.49645126]


CONTENT BASE

In [103]:
dataset_anime_content_base = pd.read_csv('/content/gdrive/MyDrive/Kaggle/anime.csv')
dataset_anime_content_base.dropna()


anime_id                                               name  \
0         32281                                     Kimi no Na wa.   
1          5114                   Fullmetal Alchemist: Brotherhood   
2         28977                                           Gintama°   
3          9253                                        Steins;Gate   
4          9969                                      Gintama&#039;   
...         ...                                                ...   
12289      9316       Toushindai My Lover: Minami tai Mecha-Minami   
12290      5543                                        Under World   
12291      5621                     Violence Gekiga David no Hoshi   
12292      6133  Violence Gekiga Shin David no Hoshi: Inma Dens...   
12293     26081                   Yasuji no Pornorama: Yacchimae!!   

                                                   genre   type episodes  \
0                   Drama, Romance, School, Supernatural  Movie        1   
1      Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64   
2      Action, Comedy, Historical, Parody, Samurai, S...     TV       51   
3                                       Sci-Fi, Thriller     TV       24   
4      Action, Comedy, Historical, Parody, Samurai, S...     TV       51   
...                                                  ...    ...      ...   
12289                                             Hentai    OVA        1   
12290                                             Hentai    OVA        1   
12291                                             Hentai    OVA        4   
12292                                             Hentai    OVA        1   
12293                                             Hentai  Movie        1   

       rating  members  
0        9.37   200630  
1        9.26   793665  
2        9.25   114262  
3        9.17   673572  
4        9.16   151266  
...       ...      ...  
12289    4.15      211  
12290    4.28      183  
12291    4.88      219  
12292    4.98      175  
12293    5.46      142  

[12017 rows x 7 columns]

In [104]:
dataset_anime_content_base['type_and_genre'] = dataset_anime_content_base['type'].str.cat(dataset_anime_content_base.genre,sep=' ')
dataset_anime_content_base['anime_id'] = dataset_anime_content_base['anime_id'].unique()
dataset_anime_content_base['type_and_genre'].fillna('', inplace=True)


In [105]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(dataset_anime_content_base.type_and_genre)

In [106]:
cosine_sim_anime = cosine_similarity(tfidf_matrix)

In [108]:
cosine_sim_anime_df = pd.DataFrame(cosine_sim_anime,columns=dataset_anime.name,index=dataset_anime.name)


- Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
- Dataframe diubah menjadi numpy
- Range(start, stop, step)

In [109]:
def anime_recommendations(anime_name, similarity_data=cosine_sim_anime_df, items=dataset_anime[['name', 'genre']], k=5):

    index = similarity_data.loc[:,anime_name].to_numpy().argpartition(
        range(-1, -k, -1))

    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    closest = closest.drop(anime_name, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

In [110]:
dataset_anime[dataset_anime.name.str.contains('zoro')]

anime_id                            name                       genre  \
3444     13247  Lupin III: Lupin Ikka Seizoroi  Adventure, Comedy, Shounen   

         type episodes  rating  members                      type_and_genre  
3444  Special        1    6.99     1462  Special Adventure, Comedy, Shounen

In [111]:
anime_recommendations("Lupin III: Lupin Ikka Seizoroi")

name  \
0                 City Hunter: Goodbye My Sweetheart   
1                            Lupin III: The Last Job   
2           Lupin III: Otakara Henkyaku Daisakusen!!   
3                                         Lupin VIII   
4  Lupin III: Sweet Lost Night - Mahou no Lamp wa...   

                        genre  
0  Adventure, Comedy, Shounen  
1  Adventure, Comedy, Shounen  
2  Adventure, Comedy, Shounen  
3  Adventure, Comedy, Shounen  
4  Adventure, Comedy, Shounen

In [120]:
print(animeid_to_animeid_encoded)

{8074: 0, 11617: 1, 11757: 2, 15451: 3, 11771: 4, 20: 5, 154: 6, 170: 7, 199: 8, 225: 9, 341: 10, 430: 11, 527: 12, 552: 13, 813: 14, 1119: 15, 1121: 16, 1122: 17, 1132: 18, 1292: 19, 1313: 20, 1526: 21, 1535: 22, 1564: 23, 1689: 24, 1764: 25, 1943: 26, 2201: 27, 2404: 28, 2847: 29, 3588: 30, 4026: 31, 5114: 32, 5231: 33, 6178: 34, 6702: 35, 6880: 36, 7695: 37, 9107: 38, 9135: 39, 9760: 40, 9917: 41, 9919: 42, 9989: 43, 10408: 44, 10507: 45, 11111: 46, 11703: 47, 11737: 48, 11759: 49, 12671: 50, 14075: 51, 14093: 52, 14345: 53, 14513: 54, 16498: 55, 16512: 56, 16782: 57, 16894: 58, 16918: 59, 17265: 60, 18097: 61, 18115: 62, 18393: 63, 19315: 64, 19815: 65, 20021: 66, 20159: 67, 20507: 68, 20583: 69, 21507: 70, 21881: 71, 22199: 72, 22297: 73, 22319: 74, 22547: 75, 22729: 76, 23301: 77, 23321: 78, 23333: 79, 23755: 80, 24415: 81, 26243: 82, 27631: 83, 27899: 84, 28121: 85, 28171: 86, 28223: 87, 28497: 88, 28701: 89, 28891: 90, 29854: 91, 31043: 92, 31859: 93, 6: 94, 15: 95, 17: 96, 18: